In [1]:
import os

import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [19]:
location = './All_Data/Urban Local Bodies & Municipal Councils'

In [25]:
inda = pd.read_csv(location + '/ULB1A.csv')
indb = pd.read_csv(location + '/ULB1B.csv')

In [26]:

hrs = list()
hrs.append(1)
for i in range(1, len(indb)):
    hrs.append(round((indb['time'][i] - indb['time'][i - 1]) / (3600000) + hrs[i-1]))
    
indb['hrs'] = hrs

hrs = list()
hrs.append(1)
for i in range(1, len(inda)):
    hrs.append(round((inda['time'][i] - inda['time'][i - 1]) / (3600000) + hrs[i-1]))

inda['hrs'] = hrs    

j = 0
m=len(indb)
n=len(inda)
print(len(indb),len(inda))
for i in range(len(inda) - 1):
    if (abs(inda['hrs'][i] - indb['hrs'][j]) != 0) and ((abs(inda['hrs'][i] - indb['hrs'][j])) == 2):
        if(n>m):
            line1 = pd.DataFrame({"id": -30, "time": -30, 'vol':-1, 
                             'ph': 1000, 'solids':1000, 
                              'hardness': 1000, 'oil': 1000,
                              'bod': 1000, 'quality': 'dont know', 'hrs': inda['hrs'][i]}, index=[j])
            line2 = pd.DataFrame({"id": -30, "time": -30, 'vol': -1, 
                             'ph':1000, 'solids': 1000, 
                              'hardness': 1000, 'oil': 1000,
                              'bod': 1000, 'quality': 'dont know', 
                              'hrs': inda['hrs'][i+1]}, index=[j+1])
        
            indb = pd.concat([indb.ix[:j-1], line1, line2, indb.ix[j:]]).reset_index(drop=True)
        
    j += 1
print(len(indb), len(inda))

5201 5225


/home/sreerag/.local/lib/python3.5/site-packages/ipykernel_launcher.py:33: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/home/sreerag/.local/lib/python3.5/site-packages/ipykernel_launcher.py:33: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




5225 5225


# Autoencoder

In [32]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 1),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [28]:
num_epochs = 100
batch_size = 2
learning_rate = 1e-3
print(len(indb),indb['vol'].shape)
indbvol=np.array(indb['vol'])
dataset=np.reshape(indbvol,(len(indbvol),1))
#dataset = [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]]
#dataset = (dataset - np.min(dataset))/(np.max(dataset) - np.min(dataset))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        data = data.float()
        #print(data)
        data = Variable(data)
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data))
    #if epoch % 10 == 0:
    #    pic = to_img(output.cpu().data)
    #    save_image(pic, './mlp_img/image_{}.png'.format(epoch))

#torch.save(model.state_dict(), './sim_autoencoder.pth')

5225 (5225,)
epoch [1/100], loss:60822904832.0000
epoch [2/100], loss:60822904832.0000
epoch [3/100], loss:60822904832.0000
epoch [4/100], loss:60822904832.0000
epoch [5/100], loss:60822904832.0000
epoch [6/100], loss:60822904832.0000
epoch [7/100], loss:60822904832.0000
epoch [8/100], loss:60822904832.0000
epoch [9/100], loss:60822904832.0000
epoch [10/100], loss:60822904832.0000
epoch [11/100], loss:60822904832.0000
epoch [12/100], loss:60822904832.0000
epoch [13/100], loss:60822904832.0000
epoch [14/100], loss:60822904832.0000
epoch [15/100], loss:60822904832.0000
epoch [16/100], loss:60822409216.0000
epoch [17/100], loss:60822409216.0000
epoch [18/100], loss:60822409216.0000
epoch [19/100], loss:60822409216.0000
epoch [20/100], loss:60822409216.0000
epoch [21/100], loss:60822409216.0000
epoch [22/100], loss:60822409216.0000
epoch [23/100], loss:60822409216.0000
epoch [24/100], loss:60822409216.0000
epoch [25/100], loss:60822409216.0000
epoch [26/100], loss:60822409216.0000
epoch [2

In [29]:
torch.save(model.state_dict(), './sim_autoencoder.pth')

In [43]:
import pickle

In [44]:
filename = 'autoencoder.model'
pickle.dump(model, open(filename, 'wb'))

In [46]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(indb['vol'][0])
print(result)

AttributeError: 'collections.OrderedDict' object has no attribute 'predict'

In [47]:
model = torch.load('./sim_autoencoder.pth')
data = np.array([indb['vol'][0],indb['vol'][1]])
data = torch.from_numpy(data)

test_value = Variable(data.cuda())
test_value = test_value.float()
test_value = test_value.unsqueeze(0)

prediction = model(test_value)
cpu_pred = prediction.cpu()
result = cpu_pred.data.numpy()
ynew = model.predict_classes(data)
print(data)
print(ynew)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx